### Incident Analysis Demo

This notebook demonstrates how to use the **incident analysis functions** for railway delay investigation.

Available Functions:

 **`incident_view()`** - Returns tabular data of delays during an incident

 **`incident_view_heatmap_html()`** - Creates animated HTML heat maps with temporal progression

Requirements:

Run this notebook from anywhere within the RDM_analysis project

Ensure parquet data files are generated (run `python -m preprocessor.main`)

In [ ]:
# Import functions with reload to ensure latest version
import sys
import os

# Ensure project root is in Python path
project_root = os.path.dirname(os.getcwd())  # Go up from demos to RDM_analysis
if project_root not in sys.path:
    sys.path.insert(0, project_root)

try:
    import importlib
    import outputs.utils
    importlib.reload(outputs.utils)
    from outputs.utils import incident_view, incident_view_heatmap_html
    print("Successfully imported incident analysis functions")
except ImportError as e:
    print(f" Import error: {e}")
    print("Make sure you're running from the RDM_analysis directory")
except Exception as e:
    print(f" Error: {e}")

station_view function ready!
plot_station_arrivals_violin() function ready!
plot_normal_operations_violin() function ready!
station_view function ready!
plot_station_arrivals_violin() function ready!
plot_normal_operations_violin() function ready!
Successfully imported incident analysis functions


incident_view function: outputs table

In [2]:
# Example: Analyze incident 435059 for 30 minutes starting at 19:00
result_df, incident_start, analysis_period = incident_view(
    incident_code=705009 ,
    incident_date='09-AUG-2024',    # When incident occurred
    analysis_date='09-AUG-2024',    # Date to analyze
    analysis_hhmm='0800',           # Start time (08:00)
    period_minutes=300               # Duration to 13:00
)

print(f" Incident started: {incident_start}")
print(f" Analysis period: {analysis_period}")
print(f" Data shape: {result_df.shape}")
print(" Complete dataset:")
print(result_df)

Analyzing incident 705009 (started 09-AUG-2024)
Analysis period: 09-Aug-2024 08:00 to 09-Aug-2024 13:00 (300 min)
Loading 374 station files for FR
Incident Details:
  Section Code: 53221 (Hitchin)
  Incident Reason: I1
  Started: 09-AUG-2024 18:41
 Incident started: 09-AUG-2024 18:41
 Analysis period: 09-Aug-2024 08:00 to 09-Aug-2024 13:00 (300 min)
 Data shape: (50, 7)
 Complete dataset:
   STATION_CODE  PLANNED_CALLS  ACTUAL_CALLS  DELAYED_TRAINS_OUT  \
0         12931             25            25                   0   
1         13702              9             9                   0   
2         14031              7             7                   0   
3         16416             23            23                   0   
4         17132             40            40                   0   
5         18067              8             8                   0   
6         23491              3             3                   0   
7         25316             11            11                   0

Network Heatmap

In [7]:
# Example: Create network heatmap for incident 499279
incident_code = 434859
incident_date = '28-APR-2024'
analysis_date = '28-APR-2024'
analysis_hhmm = '0600' # Start at 06:00
period_minutes = 1440 # 24 hours
interval_minutes = 60

# Generate dynamic filename with all parameters
output_file = f'heatmap_incident_{incident_code}_{analysis_date.replace("-", "_")}_{analysis_hhmm}_period{period_minutes}min_interval{interval_minutes}min.html'

heatmap_html = incident_view_heatmap_html(
    incident_code=incident_code,
    incident_date=incident_date,
    analysis_date=analysis_date, 
    analysis_hhmm=analysis_hhmm,
    period_minutes=period_minutes,       # 1 hour total period
    interval_minutes=interval_minutes,   # Split into 10-minute intervals
    output_file=output_file
)

print(" Network heatmap created!")
print(f" File: {output_file}")
print("Open the HTML file in your browser to explore the full network heatmap")
print(" Shows vibrant continuous heatmap with clickable data for affected stations")

Loading station coordinates from: C:\Users\39342\University of Glasgow\Ji-Eun Byun - MZ-JB\RDM_analysis\data\reference\stations_ref_with_dft.json
Successfully loaded 6104 station records
Creating heatmap for incident 434859
Analysis period: 28-Apr-2024 06:00 to 29-Apr-2024 06:00
Interval size: 60 minutes
Total intervals: 24
Loading 366 station files for SU
Found coordinates for 385 stations (DFT categories A/B/C1/C2)
Found delay data for 21 affected stations
Incident section code: 04303:04730
Incident reason: IR
Incident start time: 28-APR-2024 02:00
Looking for STANOX codes: ['04303', '04730']
Searching for STANOX: 04303
📍 Found incident location: Edinburgh (04303)
Searching for STANOX: 04730
📍 Found incident location: Monktonhall Jn (04730)
🌡️ CONTINUOUS HEATMAP CREATED! 
File: heatmap_incident_434859_28_APR_2024_0600_period1440min_interval60min.html
Time steps: 24 (60-minute intervals)
Total stations mapped: 385 (DFT categories A/B/C1/C2)
Affected stations: 21
🔥 Features: Continuous